# Neural Networks

Read the documentation for training a simple convolutional neural network (ref. http: //www.vlfeat.org/matconvnet/training/). Train a CNN with two convolutional layers and one fully connected layer, with the architecture specified as follows: number of nodes: 20-50-500-21. The number of the nodes in the last layer is fixed as 21 as we are per- forming 21-category (20 CMU PIE faces plus 1 for yourself) classification. Convolutional kernel sizes are set as 5. Each convolutional layer is followed by a max pooling layer with a kernel size of 2 and stride of 2. The fully connected layer is followed by ReLU. Train the network and report the final classification performance.

### Read all the images

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits import mplot3d

In [2]:
def read_images():
    dir='PIE'
    PIE=os.listdir(dir)
    all_img = []
    img_folders=[dir+'/' + photo for photo in PIE]
    selected_subjects = np.random.choice(range(len(img_folders)), 20, replace=False)
    
    selected_sub_folders = [img_folders[index] for index in selected_subjects]
    print('Selected Subjects:')
    print(selected_sub_folders)

    for img_dir in selected_sub_folders:
        img_path=os.listdir(img_dir)
        img_dir_index=[img_dir+'/' + photo for photo in img_path]
        all_img.append(img_dir_index)

    flat_list = []
    for sublist in all_img:
        for item in sublist:
            flat_list.append(item)
    
    images = np.array([plt.imread(image) for image in flat_list], dtype=np.int16)
    labels = np.zeros((len(images)))

    for i, path in enumerate(flat_list):
        labels[i] = path.split('/')[1]
    return images, labels

In [3]:
def read_my_images():
    my_img_path = 'my_images'
    img_list=os.listdir(my_img_path)
    print(img_list)
    my_images = np.array([plt.imread(my_img_path + '/' + image) for image in img_list], dtype=np.int16)
    my_labels = np.full((10), 69)
    return my_images, my_labels

In [4]:
images, labels = read_images()
n_samples, image_size, _ = images.shape
n_features = image_size * image_size

Selected Subjects:
['PIE/64', 'PIE/8', 'PIE/61', 'PIE/47', 'PIE/14', 'PIE/38', 'PIE/58', 'PIE/30', 'PIE/24', 'PIE/48', 'PIE/25', 'PIE/41', 'PIE/19', 'PIE/45', 'PIE/29', 'PIE/40', 'PIE/17', 'PIE/18', 'PIE/6', 'PIE/36']


In [5]:
my_images, my_labels = read_my_images()

['8.jpg', '9.jpg', '4.jpg', '5.jpg', '7.jpg', '6.jpg', '2.jpg', '3.jpg', '1.jpg', '0.jpg']


In [6]:
np.unique(labels).shape

(20,)

### Randomly select 500 images

In [7]:
def select_images(images, labels, select_num):
    num_images = len(images)
    np.random.seed(99)
    if select_num == len(images):
        return images.reshape([select_num, n_features]), labels
    rand_images_list = np.random.randint(0, num_images, select_num)
    X = images[rand_images_list].reshape([select_num, n_features])
    y = labels[rand_images_list]
    return X, y

In [8]:
X, y = select_images(images, labels, 500)

In [9]:
np.unique(labels).shape

(20,)

## Convolutional Neural Network

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras import optimizers
import keras.backend as K

Using TensorFlow backend.


In [11]:
X, y = select_images(images, labels, n_samples)

In [12]:
X = X.reshape(-1, image_size, image_size, 1)
my_images = my_images.reshape(-1, image_size, image_size, 1)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [14]:
X_train.shape

(2375, 32, 32, 1)

In [15]:
my_img_train = np.random.choice(range(10), 7, replace=False)
my_img_test = np.delete(np.arange(0,10), my_img_train)

In [16]:
X_train_self = my_images[my_img_train]
X_test_self = my_images[my_img_test]
y_train_self = my_labels[my_img_train]
y_test_self = my_labels[my_img_test]

In [17]:
X_train = np.concatenate((X_train, X_train_self), axis=0)
X_test = np.concatenate((X_test, X_test_self), axis=0)
y_train = np.concatenate((y_train, y_train_self), axis=0)
y_test = np.concatenate((y_test, y_test_self), axis=0)

In [18]:
# binary encode
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
y_train_reshaped = y_train.reshape(len(y_train), 1)
y_train_onehot_encoded = onehot_encoder.fit_transform(y_train_reshaped)
y_test_reshaped = y_test.reshape(len(y_test), 1)
y_test_onehot_encoded = onehot_encoder.fit_transform(y_test_reshaped)

In [19]:
K.clear_session()

model = Sequential()

model.add(Conv2D(20, (5, 5), activation='relu', input_shape=(image_size, image_size, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(50, (5, 5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(500, activation='relu'))

model.add(Dense(21, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),  
              metrics=['accuracy'])

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 20)        520       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 50)        25050     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1250)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               625500    
_________________________________________________________________
dense_2 (Dense)              (None, 21)                10521     
Total para

In [21]:
model.fit(X_train, y_train_onehot_encoded, batch_size=128,
          epochs=50, verbose=1, validation_split=0.3)

Train on 1667 samples, validate on 715 samples
Epoch 1/50
1667/1667 [==============================] - 12s 7ms/step - loss: 11.7218 - acc: 0.0672 - val_loss: 11.1339 - val_acc: 0.0643
Epoch 2/50
1667/1667 [==============================] - 12s 7ms/step - loss: 5.4749 - acc: 0.1752 - val_loss: 6.9730 - val_acc: 0.0713
Epoch 3/50
1667/1667 [==============================] - 12s 7ms/step - loss: 3.3614 - acc: 0.2825 - val_loss: 8.3418 - val_acc: 0.0503
Epoch 4/50
1667/1667 [==============================] - 11s 7ms/step - loss: 2.8417 - acc: 0.3959 - val_loss: 4.4992 - val_acc: 0.1664
Epoch 5/50
1667/1667 [==============================] - 11s 7ms/step - loss: 1.9293 - acc: 0.4979 - val_loss: 6.2428 - val_acc: 0.1399
Epoch 6/50
1667/1667 [==============================] - 11s 7ms/step - loss: 1.6192 - acc: 0.5867 - val_loss: 3.0092 - val_acc: 0.2853
Epoch 7/50
1667/1667 [==============================] - 11s 7ms/step - loss: 1.1439 - acc: 0.6689 - val_loss: 5.3090 - val_acc: 0.1427
Epoch 

In [22]:
model.evaluate(X_test, y_test_onehot_encoded)

1022/1022 [==============================] - 2s 2ms/step


[0.26705302254327823, 0.948140900428981]

In [27]:
y_pred = model.predict(X_test)

In [28]:
np.argmax(y_pred[-3:], axis=1)

array([13,  2,  0])

In [29]:
y_pred = model.predict(X_train)

In [30]:
np.argmax(y_pred[-7:], axis=1)

array([0, 0, 0, 0, 4, 0, 0])